In [3]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

### Read dataset


In [4]:
sales_reciepts = pd.read_csv("dataset/201904 sales reciepts.csv")
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [5]:
product = pd.read_csv("dataset/product.csv")
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


### Data Wrangling

#### Merge data

In [6]:
sales_reciepts = sales_reciepts[["transaction_id", "transaction_date", "sales_outlet_id", "customer_id", "product_id", "quantity"]]
product = product[["product_id", "product_category", "product"]]

dataset = pd.merge(sales_reciepts, product, on="product_id", how="left")
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


#### Remove sizes

In [7]:
dataset[dataset["product"].str.contains("Dark chocolate")]["product"].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [8]:
dataset["product"].nunique()

80

In [9]:
dataset["product"] = dataset["product"].str.replace("Rg", "")
dataset["product"] = dataset["product"].str.replace("Sm", "")
dataset["product"] = dataset["product"].str.replace("Lg", "")

In [10]:
dataset["product"].nunique()

60

In [11]:
print(dataset["product"].unique())

['Traditional Blend Chai ' 'Brazilian ' 'Serenity Green Tea '
 'Our Old Time Diner Blend ' 'Jamaican Coffee River ' 'Ethiopia '
 'English Breakfast ' 'Sustainably Grown Organic ' 'Earl Grey '
 'Cappuccino' 'Espresso shot' 'Cappuccino ' 'Latte' 'Dark chocolate '
 'Columbian Medium Roast ' 'Oatmeal Scone' 'Morning Sunrise Chai '
 'Peppermint ' 'Jumbo Savory Scone' 'Lemon Grass ' 'Latte '
 'Chocolate Chip Biscotti' 'Spicy Eye Opener Chai ' 'Ginger Biscotti'
 'Chocolate Croissant' 'Hazelnut Biscotti' 'Cranberry Scone'
 'Scottish Cream Scone ' 'Croissant' 'Almond Croissant' 'Ginger Scone'
 'Ouro Brasileiro shot' 'Organic Decaf Blend' 'Chocolate syrup'
 'Peppermint' 'Ethiopia' 'Hazelnut syrup' 'Carmel syrup'
 'English Breakfast' 'Sugar Free Vanilla syrup' 'Our Old Time Diner Blend'
 'Jamacian Coffee River' 'Spicy Eye Opener Chai'
 'Sustainably Grown Organic' 'Dark chocolate'
 'Guatemalan Sustainably Grown' 'Civet Cat' 'Serenity Green Tea'
 'Chili Mayan' 'Primo Espresso Roast' 'Brazilian - Or

#### Choose Product Subset

In [12]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [13]:
dataset = dataset[dataset['product'].isin(products_to_take)]
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino
23,137,2019-04-01,3,264,40,1,Coffee,Cappuccino


In [14]:
dataset[['product','product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Oatmeal Scone,Bakery
4,Jumbo Savory Scone,Bakery
5,Chocolate Chip Biscotti,Bakery
6,Ginger Biscotti,Bakery
7,Chocolate Croissant,Bakery
8,Hazelnut Biscotti,Bakery
9,Cranberry Scone,Bakery


#### Clean Transactions

In [15]:
dataset['transaction'] = dataset['transaction_id'].astype(str) +"_"+  dataset['customer_id'].astype(str)
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131
23,137,2019-04-01,3,264,40,1,Coffee,Cappuccino,137_264


In [16]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
valid_transactions = num_of_items_for_each_transaction[(num_of_items_for_each_transaction['count']>1)]['transaction'].tolist()
dataset = dataset[dataset['transaction'].isin(valid_transactions)]

In [17]:
num_of_items_for_each_transaction.head()

,transaction,count
0,209_0,31
1,206_0,29
2,204_0,27
3,208_0,25
4,203_0,24


In [18]:
dataset.shape

(7346, 9)

#### Product Trends

In [19]:
dataset["product_category"].value_counts()

product_category
Bakery                3459
Flavours              1972
Coffee                1896
Packaged Chocolate      19
Name: count, dtype: int64

In [20]:
dataset['product'].value_counts()

product
Cappuccino                  645
Latte                       631
Espresso shot               620
Chocolate Croissant         601
Sugar Free Vanilla syrup    530
Chocolate syrup             506
Carmel syrup                486
Hazelnut syrup              450
Ginger Scone                381
Croissant                   330
Jumbo Savory Scone          319
Almond Croissant            316
Chocolate Chip Biscotti     315
Cranberry Scone             308
Hazelnut Biscotti           307
Oatmeal Scone               293
Ginger Biscotti             289
Dark chocolate               19
Name: count, dtype: int64

## Popularity Recommendation Engine

In [21]:
popularity_recommendation = dataset.groupby(['product','product_category']).count().reset_index()

In [22]:
popularity_recommendation = popularity_recommendation[['product','product_category','transaction_id']]
popularity_recommendation = popularity_recommendation.rename(columns = {'transaction_id':'number_of_transactions'})

In [23]:
popularity_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv',index=False)

## Apriori Recommendation Engine

In [ ]:
train_basket = (dataset.groupby(['transaction','product'])['product'].count().reset_index(name ='Count'))
train_basket.head(10)

# Long Format

,transaction,product,Count
0,1001_8306,Cappuccino,1
1,1001_8306,Carmel syrup,1
2,1002_0,Carmel syrup,1
3,1002_0,Espresso shot,1
4,1002_0,Ginger Scone,1
5,1004_5383,Chocolate syrup,1
6,1004_5383,Latte,1
7,1005_0,Jumbo Savory Scone,2
8,1005_0,Latte,1
9,1005_5559,Latte,1


In [ ]:
my_basket = train_basket.pivot_table(index='transaction', columns='product', values='Count', aggfunc='sum').fillna(0)
my_basket.head()

# Wide format

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
1005_5559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [31]:
def encode_units(x):
  if x <= 0:
    return 0
  if x > 0:
    return 1

my_basket_sets = my_basket.map(encode_units)
my_basket_sets.head(10)

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1005_5559,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1006_0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1007_8375,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1008_0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1009_5183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [33]:
frequent_items = apriori(my_basket_sets, min_support = 0.05, use_colnames = True)
frequent_items.head()

/Users/mpidev/miniconda3/envs/coffee-shop-env/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.140163,(Almond Croissant)
1,0.282365,(Cappuccino)
2,0.219674,(Carmel syrup)
3,0.133028,(Chocolate Chip Biscotti)
4,0.165138,(Chocolate Croissant)


In [34]:
rules_basket = association_rules(frequent_items, metric = "lift", min_threshold = 1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Carmel syrup),(Cappuccino),0.219674,0.282365,0.076453,0.348028,1.232546,0.014424,1.100714,0.241785
1,(Cappuccino),(Carmel syrup),0.282365,0.219674,0.076453,0.270758,1.232546,0.014424,1.070051,0.262907
2,(Chocolate syrup),(Cappuccino),0.223242,0.282365,0.086646,0.388128,1.374561,0.023611,1.172851,0.350811
3,(Cappuccino),(Chocolate syrup),0.282365,0.223242,0.086646,0.306859,1.374561,0.023611,1.120636,0.379713
4,(Hazelnut syrup),(Cappuccino),0.199796,0.282365,0.071356,0.357143,1.264827,0.014940,1.116321,0.261656


In [ ]:
# Save Rules Basket
rules_basket.to_pickle('rules_basket.pkl')

In [35]:
# Get Example Recommendations
rules_basket[rules_basket['antecedents']=={'Latte'}].sort_values('confidence',ascending=False).head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
26,(Latte),(Sugar Free Vanilla syrup),0.281346,0.231906,0.085627,0.304348,1.312375,0.020381,1.104135,0.331206
11,(Latte),(Carmel syrup),0.281346,0.219674,0.082569,0.293478,1.335973,0.020765,1.104462,0.349934
17,(Latte),(Chocolate syrup),0.281346,0.223242,0.074924,0.266304,1.192898,0.012116,1.058693,0.225011
25,(Latte),(Hazelnut syrup),0.281346,0.199796,0.073904,0.262681,1.314746,0.017692,1.085289,0.333118
13,(Latte),(Chocolate Croissant),0.281346,0.165138,0.051988,0.184783,1.118961,0.005527,1.024098,0.147935
19,(Latte),(Croissant),0.281346,0.141182,0.050459,0.179348,1.270326,0.010738,1.046506,0.296110


#### Save in Json Format

In [40]:
product_categories = dataset[['product','product_category']].drop_duplicates().set_index('product').to_dict()['product_category']
product_categories

{'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Cappuccino': 'Coffee',
 'Chocolate Croissant': 'Bakery',
 'Latte': 'Coffee',
 'Croissant': 'Bakery',
 'Cranberry Scone': 'Bakery',
 'Chocolate Chip Biscotti': 'Bakery',
 'Jumbo Savory Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours',
 'Dark chocolate': 'Packaged Chocolate'}

In [ ]:
recommendations_json = {}

antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents']==antecedent]
    df_rec = df_rec.sort_values('confidence',ascending=False)
    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _, row in df_rec.iterrows():
        rec_objects =row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists=True
            if already_exists:
                continue
            
            rec = {'product':rec_object, 
                   'product_category':product_categories[rec_object],
                   'confidence': row['confidence']
                  }
            recommendations_json[key].append(rec)

In [38]:
import pprint
pprint.pp(recommendations_json)

{'Carmel syrup': [{'product': 'Latte',
                   'product_category': 'Coffee',
                   'confidence': 0.37587006960556846},
                  {'product': 'Espresso shot',
                   'product_category': 'Coffee',
                   'confidence': 0.3712296983758701},
                  {'product': 'Cappuccino',
                   'product_category': 'Coffee',
                   'confidence': 0.3480278422273782}],
 'Cappuccino': [{'product': 'Sugar Free Vanilla syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.32671480144404336},
                {'product': 'Chocolate syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.3068592057761733},
                {'product': 'Carmel syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.27075812274368233},
                {'product': 'Hazelnut syrup',
                 'product_category': 'Flavours',
            

In [39]:
import json
with open('api/recommendation_objects/apriori_recommendations.json', 'w') as json_file:
    json.dump(recommendations_json, json_file) 